In [1]:
import rasterio
import numpy as np
from PIL import Image
import os
import pandas as pd
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torchvision.models as models
import math
from torch.autograd import Variable
torch.manual_seed(0)
np.random.seed(0)

In [3]:
LATITUDE_RANGE = (90, -90)
LONGITUDE_RANGE = (-180, 180)
NUM_SUBREGIONS = 64
SUBREGIONS_PER_ROW = 8  # Assume the regions are divided into 8x8 grid
SQUARE_SIZE_KM = 2  # Each square's size in kilometers
MOON_RADIUS_KM = 1737.4  # Approximate radius of Moon in kilometers

# Function to calculate degrees from kilometers
def km_to_degrees(km):
    return km / (2 * np.pi * MOON_RADIUS_KM) * 360

# Calculate the size of each subregion in degrees
lat_per_region = (LATITUDE_RANGE[1] - LATITUDE_RANGE[0]) / SUBREGIONS_PER_ROW
lon_per_region = (LONGITUDE_RANGE[1] - LONGITUDE_RANGE[0]) / SUBREGIONS_PER_ROW

# Size of a square in degrees
square_size_deg = km_to_degrees(SQUARE_SIZE_KM)

# Number of squares per subregion
num_squares_lat = abs(int(lat_per_region // square_size_deg))
num_squares_lon = abs(int(lon_per_region // square_size_deg))
def lat_long_to_pixel(lat, lon, img_width, img_height):
    """
    Converts latitude and longitude to pixel coordinates in the image.
    Assumes the image is georeferenced from (-180W, 90N) to (180E, -90S).
    """
    x = min(int((lon + 180) / 360 * img_width), img_width - 125)
    y = min(int((90 - lat) / 180 * img_height), img_height - 125)
    return x, y
epsilon = 1e-10
def center_lat_lon(lat, lon, width_km=2, height_km=2):
    """
    Calculate the latitude and longitude of the center pixel of a rectangular area on the Moon.

    Parameters:
        lat (float): Latitude of the top-left corner in degrees.
        lon (float): Longitude of the top-left corner in degrees.
        width_km (float): Width of the area in kilometers (default: 2 km).
        height_km (float): Height of the area in kilometers (default: 2 km).

    Returns:
        tuple: (center_lat, center_lon) - Latitude and longitude of the center in degrees.
    """
    # Average radius of the Moon in kilometers
    moon_radius_km = 1737.4

    # Convert distance to degrees for latitude
    km_per_degree_lat = (2 * math.pi * moon_radius_km) / 360
    delta_lat = (height_km / 2) / km_per_degree_lat
    if abs(lat) == 90:
        delta_lon = 0
    else:
        km_per_degree_lon = km_per_degree_lat * math.cos(math.radians(lat))
        delta_lon = (width_km / 2) / km_per_degree_lon
    center_lat = lat - delta_lat 
    center_lon = lon + delta_lon

    return center_lat, center_lon


In [4]:
def extract_sub_image(image_array, img_width, img_height, lat, lon, height_km, width_km, resolution_km):
    """
    Extracts a sub-image from the given image array and saves it as PNG in the specified folder.
    Calculates the center latitude and longitude of the extracted sub-image.
    """
    # Calculate the dimensions of the sub-image in pixels
    height_px = int(height_km / resolution_km)
    width_px = int(width_km / resolution_km)
    ul_x, ul_y = lat_long_to_pixel(lat, lon, img_width, img_height)
    
    sub_image_array = image_array[
        ul_y : ul_y + height_px, ul_x : ul_x + width_px
    ]
    center_lat, center_lon = center_lat_lon(lat, lon, width_km=2, height_km=2)

    return center_lat, center_lon, sub_image_array


In [5]:
tiff_image_path = "Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif"
output_folder = "output_images"
os.makedirs(output_folder, exist_ok=True)
with rasterio.open(tiff_image_path) as dataset:
    img_width = dataset.width
    img_height = dataset.height
    resolution_km = 0.1  # 100m/px corresponds to 0.1km/px
    image_array = dataset.read(1)   

In [8]:
resnet50 = models.resnet50(weights="ResNet50_Weights.DEFAULT")
modules=list(resnet50.children())[:-1]
resnet50 = nn.Sequential(*modules)
for p in resnet50.parameters():
    p.requires_grad = False

In [10]:

RegionWiseRow1Images = {}
RegionWiseRow1Features = {}
for i in range(SUBREGIONS_PER_ROW):
    for j in range(SUBREGIONS_PER_ROW):
        torch.cuda.empty_cache()
        subregion_lat = LATITUDE_RANGE[0] + i * lat_per_region
        subregion_lon = LONGITUDE_RANGE[0] + j * lon_per_region
        images = []
        features = []
        for lat_idx in range(1):
            for lon_idx in range(num_squares_lon):
                square_lat = subregion_lat - lat_idx * square_size_deg
                square_lon = subregion_lon + lon_idx * square_size_deg
                height_km = 12.5
                width_km = 12.5
                _, _, tempImage = extract_sub_image(image_array, img_width,
                                                    img_height, square_lat,
                                                    square_lon, height_km,
                                                    width_km, resolution_km)
                # tempImage has shape: (1, 125, 125)
                images.append(tempImage)
        # I need to combine using 
        # Now for each subregion I have a list of length 682 with images (<class 'PIL.Image.Image'>) of shape (1,125,125)
        # Using this I need to make a Tensor of shape (682, 1, 125, 125)
        # Transformation to convert PIL Image to Tensor
        target_size = (125, 125)
        transform = transforms.Compose([
            transforms.ToTensor(),           # Converts to Tensor with shape (C, H, W)
            transforms.Resize(target_size),  # Resize all images to (125, 125)
        ])
        # Convert each image to a Tensor and add to a list
        tensors = [transform(img) for img in images]
        # Stack tensors to create a single tensor with shape (682, 1, 125, 125)
        tensor_stack = torch.stack(tensors)
        
        # resnet50 = resnet50.to('cuda')
        # features = resnet50(Variable(tensor_stack.repeat(1, 3, 1, 1).normal_().to('cuda'))).data
        resnet50 = resnet50.to('cuda')
        features_list = []
        for batch_start in range(0, len(tensor_stack), 32):
            batch = tensor_stack[batch_start:batch_start + 32]
            batch = batch.repeat(1, 3, 1, 1).normal_().to('cuda')  # Repeat to match input channels
            with torch.no_grad():  # Avoid computing gradients to save memory
                batch_features = resnet50(Variable(batch)).data
            features_list.append(batch_features)
            torch.cuda.empty_cache()  # Clear cache after each batch

        # Combine all features
        features = torch.cat(features_list, dim=0)

        print("Done:", i, j)
        RegionWiseRow1Images[f"{i}_{j}"] = tensor_stack
        RegionWiseRow1Features[f"{i}_{j}"] = features
        
        del tensor_stack
        del features

Done: 0 0
Done: 0 1
Done: 0 2
Done: 0 3
Done: 0 4
Done: 0 5
Done: 0 6
Done: 0 7
Done: 1 0
Done: 1 1
Done: 1 2
Done: 1 3
Done: 1 4
Done: 1 5
Done: 1 6
Done: 1 7
Done: 2 0
Done: 2 1
Done: 2 2
Done: 2 3
Done: 2 4
Done: 2 5
Done: 2 6
Done: 2 7
Done: 3 0
Done: 3 1
Done: 3 2
Done: 3 3
Done: 3 4
Done: 3 5
Done: 3 6
Done: 3 7
Done: 4 0
Done: 4 1
Done: 4 2
Done: 4 3
Done: 4 4
Done: 4 5
Done: 4 6
Done: 4 7
Done: 5 0
Done: 5 1
Done: 5 2
Done: 5 3
Done: 5 4
Done: 5 5
Done: 5 6
Done: 5 7
Done: 6 0
Done: 6 1
Done: 6 2
Done: 6 3
Done: 6 4
Done: 6 5
Done: 6 6
Done: 6 7
Done: 7 0
Done: 7 1
Done: 7 2
Done: 7 3
Done: 7 4
Done: 7 5
Done: 7 6
Done: 7 7


# Features with most variance

In [12]:
all_features = []
for i in range(8):
    for j in range(8):
        key = f"{i}_{j}"
        features = RegionWiseRow1Features[key].squeeze(-1).squeeze(-1)  # Shape: [682, 2048]
        all_features.append(features)

all_features = torch.cat(all_features, dim=0)  # Shape: [64 * 682, 2048]

# Step 2: Compute correlation matrix
correlation_matrix = torch.corrcoef(all_features.T)  # Shape: [2048, 2048]

# Step 3: Select features using a greedy approach
selected_features_indices = []
num_features = 300  # Number of features to select
remaining_features = list(range(2048))

while len(selected_features_indices) < num_features:
    scores = []
    for idx in remaining_features:
        relevance = correlation_matrix[idx, idx]  # Self-correlation (variance proxy)
        redundancy = sum(correlation_matrix[idx, sel] for sel in selected_features_indices) / (
            len(selected_features_indices) + 1e-8
        )
        score = relevance - redundancy
        scores.append((score, idx))

    # Select the feature with the highest score
    best_feature = max(scores, key=lambda x: x[0])[1]
    selected_features_indices.append(best_feature)
    remaining_features.remove(best_feature)

# Step 4: Extract selected features
selected_features = {}
for i in range(8):
    for j in range(8):
        key = f"{i}_{j}"
        features = RegionWiseRow1Features[key].squeeze(-1).squeeze(-1)  # Shape: [682, 2048]
        selected_features[key] = features[:, selected_features_indices]  # Shape: [682, 100]


In [13]:
print(selected_features_indices)

[0, 1846, 1808, 1813, 1146, 1378, 923, 1237, 1558, 37, 1574, 1117, 103, 505, 550, 1734, 1785, 881, 1030, 1820, 1978, 792, 1323, 51, 1714, 691, 978, 1746, 1499, 1183, 1160, 1288, 371, 985, 34, 1696, 1101, 469, 1406, 133, 703, 1679, 258, 857, 1245, 914, 184, 157, 1988, 1641, 947, 1847, 1953, 2007, 787, 129, 793, 188, 163, 1262, 800, 1131, 1390, 66, 700, 590, 662, 916, 1538, 1673, 995, 1424, 139, 652, 959, 1869, 228, 1293, 1105, 1457, 2015, 692, 149, 1958, 647, 1530, 1228, 930, 567, 1003, 46, 1341, 1045, 1560, 741, 1995, 522, 1728, 1298, 783, 1778, 1077, 640, 1774, 226, 1694, 285, 969, 97, 1863, 578, 558, 780, 813, 397, 643, 696, 1026, 434, 559, 1699, 1195, 251, 534, 555, 1555, 1676, 403, 1373, 577, 762, 912, 1611, 943, 278, 1135, 1584, 1207, 323, 186, 1076, 1470, 1564, 952, 221, 1184, 419, 478, 880, 1276, 1938, 982, 1159, 116, 395, 1936, 1926, 980, 729, 524, 1290, 252, 1670, 264, 1727, 1083, 412, 398, 1155, 814, 688, 1865, 126, 561, 1835, 1372, 1154, 716, 362, 216, 1534, 320, 463, 866, 9